<a href="https://colab.research.google.com/github/CharlesColgan/MSDS-6381-CODE/blob/main/Project/6381_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Packages

In [286]:
from google.colab import files
import time
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data Sets

https://www.kaggle.com/sl6149/data-scientist-job-market-in-the-us

import

In [287]:
header = ["Position", "Company", "Description", "Number of Reviews", "City"]

AL = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeAL.csv", names = header).drop(0).reset_index(drop=True)

AT = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeAT.csv", names = header).drop(0).reset_index(drop=True)

BO = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeBO.csv", names = header).drop(0).reset_index(drop=True)

BOS = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeBOS.csv", names = header).drop(0).reset_index(drop=True)

CHI = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeCHI.csv", names = header).drop(0).reset_index(drop=True)

DC = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeDC.csv", names = header).drop(0).reset_index(drop=True)

LA = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeLA.csv", names = header).drop(0).reset_index(drop=True)

MA = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeMA.csv", names = header).drop(0).reset_index(drop=True)

MV = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeMV.csv", names = header).drop(0).reset_index(drop=True)

NY = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeNY.csv", names = header).drop(0).reset_index(drop=True)

RM = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeRM.csv", names = header).drop(0).reset_index(drop=True)

SD = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeSD.csv", names = header).drop(0).reset_index(drop=True)

SEA = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeSEA.csv", names = header).drop(0).reset_index(drop=True)

SF = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeSF.csv", names = header).drop(0).reset_index(drop=True)

US = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/fulltimeSU.csv", names = header).drop(0).reset_index(drop=True)

Add State

In [288]:
AL["State"] = "GA"

AT["State"] = "TX"

BO["State"] = "CO"

BOS["State"] = "MA"

CHI["State"] = "IL"

DC["State"] = "DC"

LA["State"] = "CA"

MA["State"] = "MA"

MV["State"] = "CA"

NY["State"] = "NY"

RM["State"] = "WA"

SD["State"] = "CA"

SEA["State"] = "WA"

SF["State"] = "CA"

US["State"] = "CA"

Combine into single DataFrame

In [289]:
ALL = pd.concat([AL,AT,BO,BOS,CHI,DC,LA,MA,MV,NY,RM,SD,SEA,SF,US]).reset_index(drop=True)

ALL = ALL.dropna().reset_index(drop=True)

Create Copy

In [290]:
ALL_BASE = ALL.copy()

# Cleaning

identifiy stings to remove

In [291]:
def clean_desc(text):

  def remove_html_tags(text):
    "Remove html tags from string"

    clean = re.compile("<.*?>")

    return re.sub(clean, " ", text)

  def n_strip(text):
    "Remove \n from string"

    clean = re.compile("\n")

    return re.sub(clean, " ", text)

  def W_strip(text):
    "Remove punctuation"

    clean = re.compile("[^\w\s]")

    return re.sub(clean, "", text)

  a = remove_html_tags(text)

  b = n_strip(a)

  c = W_strip(b)

  return c

def small_clean(text):

  def remove_html_tags(text):
    "Remove html tags from string"

    clean = re.compile("<.*?>")

    return re.sub(clean, " ", text)

  def n_strip(text):
    "Remove \n from string"

    clean = re.compile("\n")

    return re.sub(clean, " ", text)

  def W_strip(text):
    "Remove punctuation"

    clean = re.compile("[^\w\s]")

    return re.sub(clean, "", text)

  a = remove_html_tags(text)

  b = n_strip(a)

  return b

def clean_rev(text):
  "Remove non-numeric from numreview"

  clean = re.compile("reviews")

  return re.sub(clean, " ", text)

def clean_cit(text):
  "Remove address and state from City"

  split = text.split(",", 1)


  return str(split[0])


Clean DataFrame

In [292]:
ALL["Description"] = ALL["Description"].apply(clean_desc).apply(str.casefold)

ALL_BASE["Description"] = ALL_BASE["Description"].apply(small_clean)

ALL["Number of Reviews"] = ALL["Number of Reviews"].apply(clean_rev)

ALL["City"] = ALL["City"].apply(clean_cit)

ALL["Position"] = ALL["Position"].apply(str.casefold)

# Catagorizing

Create Catagories

In [293]:
CAT = pd.read_csv("https://raw.githubusercontent.com/CharlesColgan/MSDS-6381-CODE/main/Project/categories.csv")

Exp = list(CAT["year of experience"].dropna())

Deg = [str.lower(x) for x in list(CAT["degrees"].dropna())]

Skill = [str.lower(x) for x in list(CAT["skills"].dropna())]

Title = ["analyst", "engineer", "scientist", "senior", "management", "specialist", "assistant"]

Create Labeler

In [294]:
def labeler(obj, labels, simple = True):

  def scanner(obj, target):

    return target in obj 

  lab = []

  for i in range(len(labels)):

    if scanner(obj, labels[i]) == True:

      lab.append(labels[i])

  if simple == True:

    if len(lab) > 1:

      lab.clear()

      lab = ["multiple"]

      return lab

    else:

      return lab

  else:

    return lab

def string_form(text): 
    
    str1 = " " 

    if type(text) != str:

      return (str1.join(text))

    else:

      return ("other")

Find/apply correct labels

In [295]:
a, b, c, d = [], [], [], []

for i in range(len(ALL["Position"])):

  a.append(labeler(ALL["Position"][i], Title))

  b.append(labeler(ALL["Description"][i], Deg))

  c.append(labeler(ALL["Description"][i], Skill, simple = False))

ALL["Title"] = a; ALL["Degree"] = b; ALL["Skills"] = c

ALL["Title"] = ALL["Title"].apply(lambda y: np.nan if len(y)==0 else y); ALL["Title"] = ALL["Title"].fillna("other")

ALL["Degree"] = ALL["Degree"].apply(lambda y: np.nan if len(y)==0 else y); ALL["Degree"] = ALL["Degree"].fillna("other")

ALL["Skills"] = ALL["Skills"].apply(lambda y: np.nan if len(y)==0 else y)

ALL["Title"] = ALL["Title"].fillna("other"); ALL["Degree"] = ALL["Degree"].fillna("other"); ALL["Skills"] = ALL["Skills"].fillna("other")

ALL["Title"] = ALL["Title"].apply(string_form); ALL["Degree"] = ALL["Degree"].apply(string_form); ALL["Skills"] = ALL["Skills"].apply(len)

ALL["Number of Reviews"] = ALL["Number of Reviews"].fillna(0)

Title.append("other")

Deg.append("other")

Skill.append("other")

In [296]:
ALL["Number of Reviews"][7]

'38,844  '

# Compiling for Print

Reset column formats

In [297]:
ALL["Position"] = ALL_BASE["Position"]

ALL["Description"] = ALL_BASE["Description"]

Write to Excel

In [298]:
ALL.to_excel("ALL.xlsx")

files.download("ALL.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Natrual Feature Genrator

tokenize

In [299]:
ALL["Description"] = ALL["Description"].apply(word_tokenize)

Remove excess words

In [300]:
locations = list(set(ALL["City"].tolist()))

stop_words = stopwords.words("english")

indexes = list(ALL.columns)

companies = list(set(ALL["Company"].tolist()))

strip_words = stop_words+locations+indexes+companies

for i in range(len(ALL)):

  ALL["Description"][i] = (" ").join([word for word in ALL["Description"][i] if not word in strip_words])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


TF-IDF

In [301]:
def get_uniques(text):

  tfIdfVectorizer=TfidfVectorizer(use_idf=True)

  tfIdf = tfIdfVectorizer.fit_transform(text)

  df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns = ["TF-IDF"])

  uniques = list(set(df[df["TF-IDF"] == 0].index))

  return uniques

def stripper(text, strip):

  token = text.apply(word_tokenize)

  for j in range(len(text)):

    token[j] = (" ").join([word for word in token[j] if not word in strip])

  return token

def commoniser(text, passes = 5):

  scanned  = stripper(text, get_uniques(text))

  if passes == 0:

    return scanned

  else:

    return commoniser(scanned, passes  = passes - 1)

# New Section